In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from typing import Tuple, Optional

import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

import os
import sys
import tarfile
import hashlib
import re
import glob

import random
import math

import IPython.display as ipd
from tensorflow.python.util import compat

In [3]:
LIB_PATH = '/content/drive/MyDrive/GSC/GSC_helper'
sys.path.append(LIB_PATH)
from utils import _download, unzipzip, zipzip
from GSC import download_GSC
from GSC12 import SpeechCommands12

## GSC

In [8]:
ZIP_MAP = download_GSC('https://drive.google.com/file/d/1nvubIeIQ4K7cte-7JRIKpaXJ0rW4FoCf/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-5Bglq0ihfzZ4tM_zZ6dxAsnoYv1YzAm/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-6St6zZqFvDy0JHhHTNV9UORH0mTjoDp/view?usp=drive_link',
                       '/content/GSC_8',
                       end = '.zip')
CSV_MAP = download_GSC('https://drive.google.com/file/d/1-GQ55fRsP1zNat93Yoc2tM7tUqUPiuch/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-FIGYCKCiKdTqZjcdbqVdKTUBt6-R1w9/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-Mpiyo-AE5at9ahnO8qD9ljl75lr7DJA/view?usp=drive_link',
                       '/content/GSC_8',
                       end = '.csv')

Downloading...
From (original): https://drive.google.com/uc?id=1nvubIeIQ4K7cte-7JRIKpaXJ0rW4FoCf
From (redirected): https://drive.google.com/uc?id=1nvubIeIQ4K7cte-7JRIKpaXJ0rW4FoCf&confirm=t&uuid=c0b03bd2-6dac-406e-82d0-bb78c0b4d2b3
To: /content/GSC_8/train.zip
100%|██████████| 58.6M/58.6M [00:04<00:00, 12.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-5Bglq0ihfzZ4tM_zZ6dxAsnoYv1YzAm
From (redirected): https://drive.google.com/uc?id=1-5Bglq0ihfzZ4tM_zZ6dxAsnoYv1YzAm&confirm=t&uuid=e11b3475-21df-4a53-b910-d7c0afee582b
To: /content/GSC_8/val.zip
100%|██████████| 5.66M/5.66M [00:00<00:00, 16.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-6St6zZqFvDy0JHhHTNV9UORH0mTjoDp
To: /content/GSC_8/test.zip
100%|██████████| 6.34M/6.34M [00:00<00:00, 14.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-GQ55fRsP1zNat93Yoc2tM7tUqUPiuch
To: /content/GSC_8/train.csv
100%|██████████| 600k/600k [00:00<00:00, 90.1MB/s]
Downloading...
From: https://drive.goog

In [9]:
import pandas as pd
import os

class GSC_8(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,
                 root: str,
                 zip_map: dict,
                 csv_map: dict,
                 unzip: bool = True,
                 subset: str = 'train',
                 batch_size: int = 32,
                 shuffle: bool = True):
        'Initialization'
        super().__init__()
        local_path = os.path.join(root, subset)
        self.root = root
        if not os.path.exists(local_path):
            os.mkdir(local_path)
            unzipzip(zip_map[subset], local_path)
        if unzip:
            unzipzip(zip_map[subset], local_path)
        self.csv = pd.read_csv(csv_map[subset])
        self.subset = subset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the numbber of batches per epoch'
        return int(np.floor(len(self.csv)/self.batch_size))

    def __getitem__(self, index):
        'Generate on batch of data'
        # Generate indexes of the batcch
        indexes = self.indexes[index*self.batch_size: (index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Undates indexes after each epoch'
        self.indexes = np.arange(len(self.csv))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X: (n_samples, *dim, n_channels)
        # Initialization
        X = []
        y = []

        # Generate data
        for i in indexes:
            # Store sample
            row = self.csv.iloc[i]
            X.append(np.load(os.path.join(self.root, row['link']))['arr_0'])

            # Store class
            y.append(row['label'])
        X = np.stack(X, axis = 0)
        y = np.stack(y, axis = 0)
        return tf.convert_to_tensor(X), tf.convert_to_tensor(y)

In [10]:
train_dataloader = GSC_8('/content/GSC_8', ZIP_MAP, CSV_MAP, unzip = False, subset = 'train', batch_size = 128, shuffle = True)
val_dataloader = GSC_8('/content/GSC_8', ZIP_MAP, CSV_MAP, unzip = False, subset = 'val', batch_size = 128, shuffle = False)
test_dataloader = GSC_8('/content/GSC_8', ZIP_MAP, CSV_MAP, unzip = False, subset = 'test', batch_size = 10, shuffle = False)

Extracted /content/GSC_8/train.zip
Extracted /content/GSC_8/val.zip
Extracted /content/GSC_8/test.zip


## BCResNet-TF

In [ ]:
X, y = next(iter(train_dataloader))
X.shape

In [31]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

class SubSpectralNorm(keras.Model):
    def __init__(self, normalized_shape, spec_groups = 16, affine = 'Sub', batch = True):
        super().__init__()
        self.spec_groups = spec_groups
        self.affine_all = False
        if affine == 'Sub':
            affine_norm = True
        else:
            assert affine == 'Sub', "Haven't supported yet"

        if batch:
            self.ssnorm = keras.layers.BatchNormalization(axis = -1)
        else:
            self.ssnorm = keras.layers.InstanceNormalization(axis = [0, 1])
        h, w, c = normalized_shape
        self.permute1 = keras.layers.Permute((3, 1, 2))
        self.reshape1 = keras.layers.Reshape((c*spec_groups, h//spec_groups, w))
        self.reshape2 = keras.layers.Reshape((c, h, w))
        self.permute2 = keras.layers.Permute((2, 3, 1))

    def call(self, x):
        """
        x: (N, F, T, C)
        """
        x = self.reshape1(self.permute1(x))
        x = self.ssnorm(x)
        x = self.permute2(self.reshape2(x))
        return x

class AdaptiveAvgPool2d(keras.Model):
    def __init__(self,
                 axis: list,
                 keepdims:bool = True) -> None:
        super().__init__()
        self.axis = axis
        self.keepdims = keepdims

    def call(self, input):
        return tf.reduce_mean(input, axis = self.axis, keepdims = self.keepdims)

from typing import Tuple, List

class BaseBlock(keras.Model):
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: Tuple[int, int],
                 stride: Tuple[int, int],
                 dilation: Tuple[int, int],
                 bias: bool = True,
                 ssn_kwargs: dict = None,
                 dropout: float = 0.1
                 ) -> None:
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        f_kernel = (1, kernel_size[0])
        t_kernel = (kernel_size[1], 1)
        f_stride = (1, stride[0])
        t_stride = (stride[1], 1)
        f_dilation = (1, dilation[0])
        t_dilation = (dilation[1], 1)

        # Freq_conv
        f2 = []
        if in_channels != out_channels:
            f2.extend([keras.layers.Conv2D(filters = out_channels,
                                           kernel_size = 1,
                                           use_bias = bias),
                        keras.layers.BatchNormalization(axis = -1),
                        keras.layers.ReLU()])


        self.f2 = keras.Sequential([*f2,
                                keras.layers.Conv2D(filters = out_channels,
                                                    kernel_size = f_kernel,
                                                    strides = f_stride,
                                                    padding = 'same',
                                                    dilation_rate = f_dilation,
                                                    groups = out_channels,
                                                    use_bias = bias
                                          ),
                                SubSpectralNorm(**ssn_kwargs)
                                if ssn_kwargs else keras.layers.BatchNormalization(axis = -1)])

        # Temporal_conv
        self.f1 = keras.Sequential([AdaptiveAvgPool2d(axis = 1, keepdims = True),
                                keras.layers.Conv2D(filters = out_channels,
                                                    kernel_size = t_kernel,
                                                    strides = t_stride,
                                                    dilation_rate = t_dilation,
                                                    padding = 'same',
                                                    groups = out_channels,
                                                    use_bias = bias,
                                                    activation = 'swish'),
                                keras.layers.BatchNormalization(axis = -1),
                                keras.layers.Activation("swish"),
                                keras.layers.Conv2D(filters = out_channels,
                                                    kernel_size = 1,
                                                    use_bias = bias),
                                keras.layers.Dropout(dropout)])
        self.relu = keras.layers.ReLU()

    def call(self,
             input: tf.Tensor) -> tf.Tensor:
        auxiliary_x = self.f2(input)
        output = self.f1(auxiliary_x)
        if self.in_channels == self.out_channels:
            output = output + auxiliary_x + input
        else:
            output = output + auxiliary_x
        return self.relu(output)

class BCResBlock(keras.Model):
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: Tuple[int, int],
                 stride: Tuple[int, int],
                 bias: bool = True,
                 ssn_kwargs: dict = None,
                 dropout: float = 0.1,
                 num_blks: int = 1,
                 idx: int = 0
                 ) -> None:
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_blks = num_blks

        blks = []
        for i in range(num_blks):
            blks.append(BaseBlock(in_channels if i==0 else out_channels,
                                  out_channels,
                                  kernel_size,
                                  stride if i == 0 else (1, 1),
                                  # dilation = (1, 2**idx),
                                  dilation = (1, 1),
                                  bias = False,
                                  ssn_kwargs = ssn_kwargs))
        self.blks = keras.Sequential(blks)

    def call(self,
                input: tf.Tensor) -> tf.Tensor:
        return self.blks(input)

class BCResNet(keras.Model):
    def __init__(self,
                 in_channels: int,
                 num_classes: int,
                 bias: bool = False,
                 mul_factor: int = 1) -> None:
        super().__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.mul_factor = mul_factor
        self.net = keras.Sequential([keras.layers.Conv2D(filters = int(16*mul_factor),
                                                     kernel_size = 5,
                                                     padding = 'same',
                                                     strides = (1, 2),
                                                     use_bias = bias),
                                 keras.layers.BatchNormalization(axis = -1),
                                 keras.layers.ReLU(),
                                 BCResBlock(int(16*mul_factor),
                                            int(8*mul_factor),
                                            kernel_size = (3, 3),
                                            stride = (1, 1),
                                            bias = bias,
                                            #ssn_kwargs = {
                                            #    'normalized_shape': (20, 100, int(8*mul_factor)),
                                            #    'spec_groups': 5
                                            #},
                                            num_blks = 2,
                                            idx = 0),
                                 BCResBlock(int(8*mul_factor),
                                            int(12*mul_factor),
                                            kernel_size = (3, 3),
                                            stride = (1, 1),
                                            bias = bias,
                                            #ssn_kwargs = {
                                            #    'normalized_shape': (10, 100, int(12*mul_factor)),
                                            #    'spec_groups': 5
                                            #},
                                            num_blks = 2,
                                            idx = 1),
                                 BCResBlock(int(12*mul_factor),
                                            int(16*mul_factor),
                                            kernel_size = (3, 3),
                                            stride = (1, 1),
                                            bias = bias,
                                            #ssn_kwargs = {
                                            #    'normalized_shape': (5, 100, int(16*mul_factor)),
                                            #    'spec_groups': 5
                                            #},
                                            num_blks = 4,
                                            idx = 2),
                                 BCResBlock(int(16*mul_factor),
                                            int(20*mul_factor),
                                            kernel_size = (3, 3),
                                            stride = (1, 1),
                                            bias = False,
                                            #ssn_kwargs = {
                                            #    'normalized_shape': (5, 100, int(20*mul_factor)),
                                            #    'spec_groups': 5
                                            #},
                                            num_blks = 4,
                                          idx = 3),
                                 keras.layers.Conv2D(filters = int(20*mul_factor),
                                                     kernel_size = 5,
                                                     padding = 'same',
                                                     use_bias  = bias,
                                                     groups = int(20*mul_factor)),
                                 keras.layers.Conv2D(filters = int(32*mul_factor),
                                                     kernel_size = 1,
                                                     use_bias = bias),
                                 keras.layers.BatchNormalization(axis = -1),
                                 keras.layers.ReLU(),
                                 AdaptiveAvgPool2d([1, 2]),
                                 keras.layers.Conv2D(filters = num_classes,
                                           kernel_size = 1),
                                 keras.layers.Flatten()]
                                 )
    def call(self, input: tf.Tensor) -> tf.Tensor:
        x = self.net(input)
        return x

In [36]:
model = keras.models.Sequential([
    keras.layers.Input(shape = (49, 40, 1)),
    keras.layers.Conv2D(filters = 3, kernel_size = 3, padding = 'same'),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.Conv2D(
        filters=8,
        kernel_size=(3, 3),
        strides=(2, 2),
        padding='valid'),
    BCResNet(1, 8, mul_factor = 1),
    keras.layers.Softmax()])

model.summary()

Model: "sequential_179"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_221 (Conv2D)         (None, 49, 40, 3)         30        
                                                                 
 zero_padding2d_1 (ZeroPadd  (None, 51, 42, 3)         0         
 ing2D)                                                          
                                                                 
 conv2d_222 (Conv2D)         (None, 25, 20, 8)         224       
                                                                 
 bc_res_net_5 (BCResNet)     (None, 8)                 11372     
                                                                 
 softmax_5 (Softmax)         (None, 8)                 0         
                                                                 
Total params: 11626 (45.41 KB)
Trainable params: 10682 (41.73 KB)
Non-trainable params: 944 (3.69 KB)
________________

In [33]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'],
)

In [14]:
EPOCHS = 1
history = model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=EPOCHS,
    #callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

 73/170 [===========>..................] - ETA: 5:10 - loss: 2.0988 - accuracy: 0.1580

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_dataloader)

292/292 [==============================] - 5s 12ms/step - loss: 0.4179 - accuracy: 0.8521


[0.41787245869636536, 0.8520547747612]

## Build the sparse model

In [15]:
!pip install tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 4.3 MB/s eta 0:00:00


In [16]:
import tensorflow_model_optimization as tfmot
import tempfile

In [38]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after after 5 epochs
end_epoch = 1

num_iterations_per_epoch = len(train_dataloader)
end_step = num_iterations_per_epoch*end_epoch

# Define parameters for pruning"
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity = 0.25,
                                                             final_sparsity = 0.75,
                                                             begin_step = 0,
                                                             end_step = end_step),
#    'pruning_policy': tfmot.sparsity.keras.PruneForLatencyOnXNNPack()
}

try:
  model_for_pruning = prune_low_magnitude(model, **pruning_params)
except ValueError as e:
  print(e)

Subclassed models are not supported currently.


In [39]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

# Cluster a whole model
clustered_model = cluster_weights(model, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

ValueError: Subclassed models are not supported currently.

## BCResNet2TFLite

In [ ]:
from tqdm import tqdm

test_specs = []
test_labels = []

for i in tqdm(range(len(test_dataloader.csv))):
    row = test_dataloader.csv.iloc[i]
    spec = np.load(os.path.join('/content/GSC_8', row['link']))['arr_0']
    test_specs.append(spec)
    test_labels.append(row['label'])

test_specs = tf.convert_to_tensor(np.stack(test_specs, axis = 0))
test_labels = tf.convert_to_tensor(np.stack(test_labels))

100%|██████████| 2925/2925 [00:01<00:00, 1701.59it/s]


In [ ]:
test_specs.shape

TensorShape([2925, 49, 40, 1])

In [ ]:
test_labels.shape

TensorShape([2925])

In [ ]:
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(test_specs).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint 8 (APIs added in r2.3)
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model_quant = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
import pathlib

tflite_models_dir = pathlib.Path("/tmp/gsc_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the dynamic range quantized model:
tflite_model_quant_file = tflite_models_dir/"gsc_tflite_bcresnet1_s_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

92672

In [ ]:
tflite_model_file = tflite_models_dir/"gsc_model_bcresnet1_s.tflite"
tflite_model_file.write_bytes(tflite_model)

86980

In [ ]:
# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_image_indices):
    global test_specs

    # Initialize the interpreter
    interpreter = tf.lite.Interpreter(model_path = str(tflite_file))
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    predictions = np.zeros((len(test_image_indices), ), dtype = int)
    for i, test_image_index in enumerate(test_image_indices):
        test_image = test_specs[test_image_index]

        # Check if the input type is quantized, the rescale input data to to uint8
        if input_details['dtype'] == np.int8:
            input_scale, input_zero_point = input_details['quantization']
            test_image = test_image/input_scale + input_zero_point

        test_image = np.expand_dims(test_image, axis = 0).astype(input_details['dtype'])
        interpreter.set_tensor(input_details['index'], test_image)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details['index'])[0]

        predictions[i] = output.argmax()

    return predictions

# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global test_specs
  global test_labels

  test_image_indices = range(test_specs.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  accuracy = (np.sum(test_labels== predictions) * 100) / len(test_specs)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_specs)))

In [ ]:
evaluate_model(tflite_model_quant_file, model_type="Quantized")

Quantized model accuracy is 79.8632% (Number of test samples=2925)


In [ ]:
evaluate_model(tflite_model_file, model_type="Float32")

Float32 model accuracy is 85.1624% (Number of test samples=2925)


In [ ]:
import shutil
shutil.copy2(str(tflite_model_quant_file), '/content')

'/content/gsc_tflite_bcresnet1_s_quant.tflite'

In [ ]:
# Save the file as a C source file
!xxd -i /content/gsc_tflite_bcresnet1_s_quant.tflite > /content/bcresnet1_s_quant.cc
# Print the source file
!cat /content/bcresnet1_s_quant.cc

unsigned char _content_gsc_tflite_bcresnet1_s_quant_tflite[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00,
  0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x00, 0x00,
  0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x84, 0x00, 0x00, 0x00, 0xdc, 0x00, 0x00, 0x00, 0xcc, 0x2d, 0x00, 0x00,
  0xdc, 0x2d, 0x00, 0x00, 0x28, 0x69, 0x01, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0xde, 0xb3, 0xff, 0xff,
  0x0c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00, 0x3c, 0x00, 0x00, 0x00,
  0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76, 0x69, 0x6e, 0x67, 0x5f,
  0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x98, 0xff, 0xff, 0xff, 0x0a, 0x01, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x09, 0x00, 0x00, 0x00, 0x73, 0x6f, 0x66, 0x74,
  0x6d, 0x61, 0x78, 0x5f, 0x31, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0xd2, 0xcf, 0xff, 0xf